# Batch Normalization
### 1) import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### 2) Checkpoint Function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### 3) Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### 4) Performance Function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) # trainig을 True로 하면 Dropout 사용
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False) # trainig을 False로 하면 Dropout 사용 X
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables) # model.variables 대신 써야 한다. 
    # batch norm 사용 시 처음에 gradient가 존재하지 않기 때문

### 5) Model Function
- Batch Normalization Function도 정의해준다.

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate):
    return tf.keras.layers.Dropout(rate)

def batch_norm():
    return tf.keras.layers.BatchNormalization()

### 6) Create Model(function version)
- dropout 함수 호출(Activation Function 다음)

In [6]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.he_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm()) # Activation Function 전에 쓴다.
        # 보통은 FC/Conv Layer - Norm - Activation 순으로 쓰거나
        # Norm - Activation - FC/Conv Layer 순으로 쓴다.
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

### 7) Define data & hyper-parameter

In [7]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### 8) Define Model & Optimizer & Writer

In [8]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batch_norm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### 9) Restore checkpoint & start train or test phase

In [9]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables)) # 여기도 variables 대신 trainable_variables로 변경

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 2.0200, train_loss: 3.08599997, train_accuracy: 0.1328, test_Accuracy: 0.1368
Epoch: [ 0] [    1/  468] time: 2.9670, train_loss: 2.95940471, train_accuracy: 0.1719, test_Accuracy: 0.1788
Epoch: [ 0] [    2/  468] time: 3.9100, train_loss: 2.95802641, train_accuracy: 0.2734, test_Accuracy: 0.2293
Epoch: [ 0] [    3/  468] time: 4.8510, train_loss: 2.73642445, train_accuracy: 0.3125, test_Accuracy: 0.2651
Epoch: [ 0] [    4/  468] time: 5.7960, train_loss: 2.88379860, train_accuracy: 0.3516, test_Accuracy: 0.2747
Epoch: [ 0] [    5/  468] time: 6.7580, train_loss: 2.84264708, train_accuracy: 0.3125, test_Accuracy: 0.2749
Epoch: [ 0] [    6/  468] time: 7.6810, train_loss: 2.38648462, train_accuracy: 0.2734, test_Accuracy: 0.2733
Epoch: [ 0] [    7/  468] time: 8.7290, train_loss: 2.64200544, train_accuracy: 0.2656, test_Accuracy: 0.2786
Epoch: [ 0] [    8/  468] time: 9.7530,

Epoch: [ 0] [   74/  468] time: 80.0270, train_loss: 0.64780682, train_accuracy: 0.8672, test_Accuracy: 0.8471
Epoch: [ 0] [   75/  468] time: 81.1110, train_loss: 0.83692259, train_accuracy: 0.8438, test_Accuracy: 0.8492
Epoch: [ 0] [   76/  468] time: 82.2250, train_loss: 0.78415585, train_accuracy: 0.8438, test_Accuracy: 0.8519
Epoch: [ 0] [   77/  468] time: 83.2860, train_loss: 0.74644947, train_accuracy: 0.8750, test_Accuracy: 0.8533
Epoch: [ 0] [   78/  468] time: 84.4840, train_loss: 0.68473423, train_accuracy: 0.8281, test_Accuracy: 0.8547
Epoch: [ 0] [   79/  468] time: 85.6970, train_loss: 0.62101716, train_accuracy: 0.8125, test_Accuracy: 0.8547
Epoch: [ 0] [   80/  468] time: 86.7940, train_loss: 0.81125563, train_accuracy: 0.8125, test_Accuracy: 0.8558
Epoch: [ 0] [   81/  468] time: 88.2030, train_loss: 0.78143823, train_accuracy: 0.7500, test_Accuracy: 0.8561
Epoch: [ 0] [   82/  468] time: 89.5700, train_loss: 0.74502528, train_accuracy: 0.8438, test_Accuracy: 0.8564
E

Epoch: [ 0] [  148/  468] time: 152.2340, train_loss: 0.55170256, train_accuracy: 0.8750, test_Accuracy: 0.9026
Epoch: [ 0] [  149/  468] time: 153.1490, train_loss: 0.53146082, train_accuracy: 0.9219, test_Accuracy: 0.9025
Epoch: [ 0] [  150/  468] time: 154.0610, train_loss: 0.67703849, train_accuracy: 0.8750, test_Accuracy: 0.9036
Epoch: [ 0] [  151/  468] time: 154.9840, train_loss: 0.43065012, train_accuracy: 0.8828, test_Accuracy: 0.9034
Epoch: [ 0] [  152/  468] time: 155.8970, train_loss: 0.43496424, train_accuracy: 0.9297, test_Accuracy: 0.9036
Epoch: [ 0] [  153/  468] time: 156.8080, train_loss: 0.45682102, train_accuracy: 0.8906, test_Accuracy: 0.9036
Epoch: [ 0] [  154/  468] time: 157.7340, train_loss: 0.44612646, train_accuracy: 0.9375, test_Accuracy: 0.9035
Epoch: [ 0] [  155/  468] time: 158.6390, train_loss: 0.61118728, train_accuracy: 0.9062, test_Accuracy: 0.9031
Epoch: [ 0] [  156/  468] time: 159.6110, train_loss: 0.51363599, train_accuracy: 0.9062, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 221.1850, train_loss: 0.29428101, train_accuracy: 0.9141, test_Accuracy: 0.9241
Epoch: [ 0] [  223/  468] time: 222.2190, train_loss: 0.30294585, train_accuracy: 0.9141, test_Accuracy: 0.9245
Epoch: [ 0] [  224/  468] time: 223.2040, train_loss: 0.41239381, train_accuracy: 0.9062, test_Accuracy: 0.9242
Epoch: [ 0] [  225/  468] time: 224.1520, train_loss: 0.38056785, train_accuracy: 0.9141, test_Accuracy: 0.9240
Epoch: [ 0] [  226/  468] time: 225.0670, train_loss: 0.44498488, train_accuracy: 0.8984, test_Accuracy: 0.9238
Epoch: [ 0] [  227/  468] time: 226.6620, train_loss: 0.39398479, train_accuracy: 0.9062, test_Accuracy: 0.9234
Epoch: [ 0] [  228/  468] time: 227.7040, train_loss: 0.34598559, train_accuracy: 0.9219, test_Accuracy: 0.9240
Epoch: [ 0] [  229/  468] time: 228.6740, train_loss: 0.46817762, train_accuracy: 0.9141, test_Accuracy: 0.9232
Epoch: [ 0] [  230/  468] time: 229.6320, train_loss: 0.42734009, train_accuracy: 0.9141, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 295.9850, train_loss: 0.32158625, train_accuracy: 0.8906, test_Accuracy: 0.9280
Epoch: [ 0] [  297/  468] time: 297.0080, train_loss: 0.40776360, train_accuracy: 0.9141, test_Accuracy: 0.9288
Epoch: [ 0] [  298/  468] time: 297.9970, train_loss: 0.37081638, train_accuracy: 0.9297, test_Accuracy: 0.9294
Epoch: [ 0] [  299/  468] time: 299.0300, train_loss: 0.37524706, train_accuracy: 0.9219, test_Accuracy: 0.9298
Epoch: [ 0] [  300/  468] time: 300.0620, train_loss: 0.58745158, train_accuracy: 0.8906, test_Accuracy: 0.9297
Epoch: [ 0] [  301/  468] time: 301.0240, train_loss: 0.38679159, train_accuracy: 0.9531, test_Accuracy: 0.9303
Epoch: [ 0] [  302/  468] time: 302.0730, train_loss: 0.37519473, train_accuracy: 0.9141, test_Accuracy: 0.9303
Epoch: [ 0] [  303/  468] time: 303.0630, train_loss: 0.36811507, train_accuracy: 0.9531, test_Accuracy: 0.9305
Epoch: [ 0] [  304/  468] time: 304.0530, train_loss: 0.26677999, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 372.2421, train_loss: 0.29227084, train_accuracy: 0.9453, test_Accuracy: 0.9386
Epoch: [ 0] [  371/  468] time: 373.3921, train_loss: 0.40142655, train_accuracy: 0.9375, test_Accuracy: 0.9390
Epoch: [ 0] [  372/  468] time: 374.5671, train_loss: 0.35455063, train_accuracy: 0.9453, test_Accuracy: 0.9400
Epoch: [ 0] [  373/  468] time: 375.7831, train_loss: 0.35427326, train_accuracy: 0.9219, test_Accuracy: 0.9409
Epoch: [ 0] [  374/  468] time: 376.8511, train_loss: 0.44320258, train_accuracy: 0.9375, test_Accuracy: 0.9416
Epoch: [ 0] [  375/  468] time: 377.7701, train_loss: 0.34867671, train_accuracy: 0.9297, test_Accuracy: 0.9420
Epoch: [ 0] [  376/  468] time: 378.7071, train_loss: 0.29911754, train_accuracy: 0.9141, test_Accuracy: 0.9425
Epoch: [ 0] [  377/  468] time: 379.6221, train_loss: 0.30546081, train_accuracy: 0.9453, test_Accuracy: 0.9424
Epoch: [ 0] [  378/  468] time: 380.6101, train_loss: 0.55485737, train_accuracy: 0.8906, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 448.1911, train_loss: 0.25590715, train_accuracy: 0.9375, test_Accuracy: 0.9437
Epoch: [ 0] [  445/  468] time: 449.1281, train_loss: 0.29291844, train_accuracy: 0.9688, test_Accuracy: 0.9440
Epoch: [ 0] [  446/  468] time: 450.0641, train_loss: 0.28541768, train_accuracy: 0.9297, test_Accuracy: 0.9438
Epoch: [ 0] [  447/  468] time: 451.0031, train_loss: 0.19306521, train_accuracy: 0.9453, test_Accuracy: 0.9443
Epoch: [ 0] [  448/  468] time: 451.9671, train_loss: 0.34481627, train_accuracy: 0.9531, test_Accuracy: 0.9445
Epoch: [ 0] [  449/  468] time: 452.9171, train_loss: 0.26042220, train_accuracy: 0.9453, test_Accuracy: 0.9449
Epoch: [ 0] [  450/  468] time: 453.8491, train_loss: 0.23925141, train_accuracy: 0.9609, test_Accuracy: 0.9447
Epoch: [ 0] [  451/  468] time: 454.8011, train_loss: 0.32224321, train_accuracy: 0.9453, test_Accuracy: 0.9445
Epoch: [ 0] [  452/  468] time: 455.7111, train_loss: 0.34509569, train_accuracy: 0.9297, test_Accuracy:

## 3. 결론
> Batch Normalization 기법을 쓰면, 좀 더 높은 accuracy의 모델을 만들 수 있다.

### Using Batch Normalization
- Test Accuracy : 94.47%